## firstly you must convert models by your system
dockerfile docker/tensorrt/Dockerfile

convertors:
* yolov5 (inference/convertors/yolo2tensorrt/bin/yolov5_tensorrt.sh)
* options (inference/convertors/options2tensorrt/convert_numberplate_options_to_onnx.py)
* ocrs (inference/convertors/ocr2tensorrt/convert_ocr_to_onnx.py)

In [1]:
import os
import matplotlib.pyplot as plt
from glob import glob
import pycuda.autoinit

from _paths import nomeroff_net_dir
from nomeroff_net.tools import unzip
from nomeroff_net.pipelines.number_plate_detection_and_reading_trt import NumberPlateDetectionAndReadingTrt

In [2]:
number_plate_detection_and_reading_trt = NumberPlateDetectionAndReadingTrt(
    task="number_plate_detection_and_reading_trt", 
    image_loader="opencv",
    path_to_model="../inference/convertors/yolo2tensorrt/yolov5s-2021-12-14.engine",
    plugin_lib="../inference/convertors/yolo2tensorrt/libmyplugins.so",
    path_to_classification_model="../data/model_repository/numberplate_options/1/model.onnx",
    prisets={
        "eu_ua_2004_2015": {
            "for_regions": ["eu_ua_2015", "eu_ua_2004"],
            "model_path": "../data/model_repository/ocr-eu_ua_2004_2015/1/model.onnx"
        },
        "eu_ua_1995": {
            "for_regions": ["eu_ua_1995"],
            "model_path": "../data/model_repository/ocr-eu_ua_1995/1/model.onnx"
        },
        "eu": {
            "for_regions": ["eu"],
            "model_path": "../data/model_repository/ocr-eu/1/model.onnx"
        },
        "ru": {
            "for_regions": ["ru", "eu-ua-ordlo-lpr", "eu-ua-ordlo-dpr"],
            "model_path": "../data/model_repository/ocr-ru/1/model.onnx"
        },
        "kz": {
            "for_regions": ["kz"],
            "model_path": "../data/model_repository/ocr-kz/1/model.onnx"
        },
        "ge": {
            "for_regions": ["ge"],
            "model_path": "../data/model_repository/ocr-ge/1/model.onnx"
        },
        "su": {
            "for_regions": ["su"],
            "model_path": "../data/model_repository/ocr-su/1/model.onnx"
        }
    },
    default_label = "eu",
    options = {
        "class_region": [
                "military",
                "eu_ua_2015",
                "eu_ua_2004",
                "eu_ua_1995",
                "eu",
                "xx_transit",
                "ru",
                "kz",
                "eu-ua-fake-dpr",
                "eu-ua-fake-lpr",
                "ge",
                "by",
                "su",
                "kg",
                "am"
            ],
            "count_lines": [
                1,
                2,
                3
            ],
    }
)

Loading weights from checkpoint (/var/www/nomeroff-net/examples/../nomeroff_net/tools/../../data/./models/np_points_craft/craft_mlt/craft_mlt_25k_2020-02-16.pth)
Loading weights of refiner from checkpoint (/var/www/nomeroff-net/examples/../nomeroff_net/tools/../../data/./models/np_points_craft/craft_refiner/craft_refiner_CTW1500_2020-02-16.pth)


In [ ]:
result = number_plate_detection_and_reading_trt(glob(os.path.join(nomeroff_net_dir,
                                                                  './data/examples/oneline_images/*')))

In [ ]:
(images, images_bboxs, 
 images_points, images_zones, region_ids, 
 region_names, count_lines, 
 confidences, texts) = unzip(result)

In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)

for img, np in zip(images, texts):
    print(np)
    plt.imshow(img)
    plt.show()